In [ ]:
import anthropic
import json

# ========== 工具定义(Tool) ==========
def calculator(expression: str) -> str:
    """计算数学表达式"""
    try:
        return str(eval(expression))
    except:
        return "计算错误"

def search_knowledge(query: str) -> str:
    """模拟RAG知识库(实际应接向量数据库)"""
    kb = {
        "台湾人口": "2300万",
        "地球半径": "6371公里"
    }
    return kb.get(query, "未找到相关信息")

TOOLS = [
    {
        "name": "calculator",
        "description": "执行数学计算,输入Python表达式",
        "input_schema": {"type": "object", "properties": {"expression": {"type": "string"}}, "required": ["expression"]}
    },
    {
        "name": "search_knowledge", 
        "description": "查询知识库",
        "input_schema": {"type": "object", "properties": {"query": {"type": "string"}}, "required": ["query"]}
    }
]

# ========== Agent主循环 ==========
def run_agent(user_goal: str, max_steps=5):
    client = anthropic.Anthropic()
    messages = [{"role": "user", "content": user_goal}]
    
    for step in range(max_steps):
        print(f"\n--- 步骤 {step+1} ---")
        
        # 1. Reasoning:让模型决策下一步
        response = client.messages.create(
            model="claude-sonnet-4-20250514",
            max_tokens=1000,
            tools=TOOLS,
            messages=messages
        )
        
        # 2. Action:执行工具或返回答案
        if response.stop_reason == "tool_use":
            tool_use = next(block for block in response.content if block.type == "tool_use")
            tool_name = tool_use.name
            tool_input = tool_use.input
            
            print(f"🔧 调用工具: {tool_name}({tool_input})")
            
            # 执行工具
            if tool_name == "calculator":
                result = calculator(tool_input["expression"])
            elif tool_name == "search_knowledge":
                result = search_knowledge(tool_input["query"])
            
            print(f"📤 工具返回: {result}")
            
            # 3. Observation:将结果反馈给模型
            messages.append({"role": "assistant", "content": response.content})
            messages.append({
                "role": "user",
                "content": [{
                    "type": "tool_result",
                    "tool_use_id": tool_use.id,
                    "content": result
                }]
            })
        else:
            # 任务完成
            final_text = next(block.text for block in response.content if hasattr(block, "text"))
            print(f"\n✅ 最终答案: {final_text}")
            return final_text
    
    return "超过最大步数限制"

# ========== 测试 ==========
run_agent("台湾人口除以100是多少?")